In [2]:
# --- Imports & setup
import os, glob, warnings
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl
from shapely.ops import unary_union
from fiona.env import Env
import pandas as pd

DATA_DIR  = r"C:\Users\krish\Desktop\SpatialCARE\DailyGPKG"
PASIG_SHP = r"C:\Users\krish\Desktop\PhD Class\Shapefile\MM\Pasig\Pasig.shp"
OUT_DIR   = r"C:\Users\krish\Desktop\SpatialCARE\Outputs\figures"
os.makedirs(OUT_DIR, exist_ok=True)

WGS84  = "+proj=longlat +datum=WGS84 +no_defs"
UTM51  = "+proj=utm +zone=51 +datum=WGS84 +units=m +no_defs"
FIG_SIZE, FIG_DPI = (7,7), 150

def dedupe_points(gdf):
    t = gdf.to_crs(UTM51).copy()
    t["x"] = t.geometry.x; t["y"] = t.geometry.y
    return t.drop_duplicates(subset=["x","y"]).to_crs(UTM51)

# --- Read Pasig
with Env(SHAPE_RESTORE_SHX="YES"):
    pasig = gpd.read_file(PASIG_SHP)
if pasig.crs is None: pasig = pasig.set_crs(WGS84)
pasig_utm = pasig.to_crs(UTM51)

# --- Collect stations from all daily files
files = sorted(glob.glob(os.path.join(DATA_DIR, "date_2025-*.gpkg")))
if not files: raise SystemExit("No daily GPKG files found.")

pts = []
for f in files:
    g = gpd.read_file(f)
    if g.crs is None: g = g.set_crs(WGS84)
    if "geometry" not in g or g.geometry.is_empty.all():
        # build geometry from lon/lat if needed
        lon = [c for c in g.columns if str(c).lower() in ("longitude","lon","x")]
        lat = [c for c in g.columns if str(c).lower() in ("latitude","lat","y")]
        if lon and lat:
            g = g.set_geometry(gpd.points_from_xy(g[lon[0]], g[lat[0]], crs=g.crs))
    pts.append(g)

all_pts = gpd.GeoDataFrame(pd.concat(pts, ignore_index=True), geometry="geometry", crs=pts[0].crs)
stations = dedupe_points(all_pts)

# --- Plot
fig, ax = plt.subplots(figsize=FIG_SIZE, dpi=FIG_DPI)
pasig_utm.boundary.plot(ax=ax, color="black", linewidth=0.8)
stations.plot(ax=ax, color="#d62728", markersize=40, edgecolor="white", linewidth=0.7)
ax.set_title("Pasig — Air Quality Stations (unique locations)")
ax.set_axis_off()
out = os.path.join(OUT_DIR, "stations_overview.png")
plt.tight_layout(); plt.savefig(out, bbox_inches="tight"); plt.close(fig)
print("Saved:", out)


Saved: C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\stations_overview.png
